In [14]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import os
import time

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão. 

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [16]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Diretório de logs

Em geral, apontamos o servidor TensorBoard para um diretório de log raiz e configuramos o código para que ele grave em um subdiretório diferente toda vez que for executado. 

Dessa forma, a mesma instância do servidor do TensorBoard permitirá que você visualize e compare dados de várias execuções do seu código, sem misturá-los.

Portanto, começamos definindo o diretório de log raiz que usaremos para armazenar os logs do TensorBoard, além de uma função que gerará um caminho do subdiretório com base na data e hora atuais, para que ele seja diferente a cada execução. 

In [17]:
root_logdir = os.path.join(os.curdir, "my_logs")
print('Diretório raiz:', root_logdir)

def get_run_logdir(root_logdir):
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir(root_logdir)
print('Subdiretório raiz:', run_logdir)

Diretório raiz: .\my_logs
Subdiretório raiz: .\my_logs\run_2022_11_03-15_31_58


### Criando e compilando o modelo.

In [18]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

### Tensorboard callback

O Keras fornece uma callback chamada **TensorBoard**, a qual passamos o caminho do subdiretório.

In [19]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

### Treinando o modelo.

In [20]:
history = model.fit(X_train, y_train, 
                    epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Epoch 1/30
363/363 [==============================] - 3s 4ms/step - loss: 1.8866 - val_loss: 0.7126
Epoch 2/30
363/363 [==============================] - 1s 3ms/step - loss: 0.6577 - val_loss: 0.6880
Epoch 3/30
363/363 [==============================] - 1s 3ms/step - loss: 0.5934 - val_loss: 0.5803
Epoch 4/30
363/363 [==============================] - 1s 3ms/step - loss: 0.5557 - val_loss: 0.5166
Epoch 5/30
363/363 [==============================] - 1s 4ms/step - loss: 0.5272 - val_loss: 0.4895
Epoch 6/30
363/363 [==============================] - 1s 3ms/step - loss: 0.5033 - val_loss: 0.4951
Epoch 7/30
363/363 [==============================] - 1s 4ms/step - loss: 0.4854 - val_loss: 0.4861
Epoch 8/30
363/363 [==============================] - 1s 4ms/step - loss: 0.4709 - val_loss: 0.4554
Epoch 9/30
363/363 [==============================] - 1s 4ms/step - loss: 0.4578 - val_loss: 0.4413
Epoch 10/30
363/363 [==============================] - 1s 4ms/step - loss: 0.4474 - val_loss: 0.4379

### Inicializando o servidor do Tensorboard

Em seguida, executamos o seguinte comando na raiz do projeto (ou de qualquer outro lugar, desde que apontemos para o diretório de log apropriado).

Para iniciar o servidor TensorBoard, uma opção é abrir um terminal. Em seguida, acessamos o diretório deste notebook e digitamos:

`$ tensorboard --logdir=./my_logs --port=6006`

Na sequência, abrimos o navegador web e o apontamos para `localhost:6006` e usamos o TensorBoard.

Como alternativa, podemos carregar a extensão Jupyter do TensorBoard e executá-la como mostrado abaixo.

In [21]:
%load_ext tensorboard

In [22]:
%tensorboard --logdir=./my_logs --port=6006

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
run_logdir2 = get_run_logdir()
run_logdir2

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=0.05))

In [ ]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir2)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

Notice how TensorBoard now sees two runs, and you can compare the learning curves.

Check out the other available logging options:

In [ ]:
help(keras.callbacks.TensorBoard.__init__)

### Referências

[1] https://towardsdatascience.com/a-quickstart-guide-to-tensorboard-fb1ade69bbcf
[2] https://neptune.ai/blog/tensorboard-tutorial